In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score



In [ ]:
# Load the IMDB dataset
max_words = 10000
max_length = 500
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_words)

In [ ]:
# Pad sequences to a maximum length of 500 words
X_train = pad_sequences(X_train, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)


In [ ]:
# Define the CNN model
def create_cnn_model():
    model = Sequential()
    model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_length))
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
# Define the CNN-LSTM model
def create_cnn_lstm_model():
    model = Sequential()
    model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_length))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01)))
    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
# Train the CNN model
cnn_model = create_cnn_model()
cnn_model.fit(X_train, y_train, batch_size=32, epochs=3, validation_data=(X_test, y_test), verbose=1)


Epoch 1/5
782/782 [==============================] - 168s 213ms/step - loss: 0.3532 - accuracy: 0.8262 - val_loss: 0.2784 - val_accuracy: 0.8838
Epoch 2/5
782/782 [==============================] - 161s 206ms/step - loss: 0.1594 - accuracy: 0.9412 - val_loss: 0.3140 - val_accuracy: 0.8813
Epoch 3/5
782/782 [==============================] - 161s 206ms/step - loss: 0.0687 - accuracy: 0.9770 - val_loss: 0.3834 - val_accuracy: 0.8818
Epoch 4/5
782/782 [==============================] - 146s 187ms/step - loss: 0.0207 - accuracy: 0.9941 - val_loss: 0.5234 - val_accuracy: 0.8799
Epoch 5/5
782/782 [==============================] - 160s 205ms/step - loss: 0.0074 - accuracy: 0.9988 - val_loss: 0.6281 - val_accuracy: 0.8768


In [ ]:
# Train the CNN-LSTM model
cnn_lstm_model = create_cnn_lstm_model()
cnn_lstm_model.fit(X_train, y_train, batch_size=32, epochs=3, validation_data=(X_test, y_test), verbose=1)


Epoch 1/3
782/782 [==============================] - 950s 1s/step - loss: 0.4862 - accuracy: 0.8201 - val_loss: 0.3739 - val_accuracy: 0.8665
Epoch 2/3
782/782 [==============================] - 945s 1s/step - loss: 0.2568 - accuracy: 0.9065 - val_loss: 0.3086 - val_accuracy: 0.8829
Epoch 3/3
782/782 [==============================] - 948s 1s/step - loss: 0.1732 - accuracy: 0.9417 - val_loss: 0.3308 - val_accuracy: 0.8744


In [ ]:
import pandas as pd
# Evaluate the models on the test set
cnn_preds = np.round(cnn_model.predict(X_test))
cnn_lstm_preds = np.round(cnn_lstm_model.predict(X_test))

cnn_acc = accuracy_score(y_test, cnn_preds)
cnn_f1 = f1_score(y_test, cnn_preds)
cnn_prec = precision_score(y_test, cnn_preds)
cnn_rec = recall_score(y_test, cnn_preds)

cnn_lstm_acc = accuracy_score(y_test, cnn_lstm_preds)
cnn_lstm_f1 = f1_score(y_test, cnn_lstm_preds)
cnn_lstm_prec = precision_score(y_test, cnn_lstm_preds)
cnn_lstm_rec = recall_score(y_test, cnn_lstm_preds)

# Create a table to compare the models
data = {'Model': ['CNN', 'CNN-LSTM'],
        'Accuracy': [cnn_acc, cnn_lstm_acc],
        'F1 Score': [cnn_f1, cnn_lstm_f1],
        'Precision': [cnn_prec, cnn_lstm_prec],
        'Recall': [cnn_rec, cnn_lstm_rec]}

df = pd.DataFrame(data)
print(df)

782/782 [==============================] - 130s 166ms/step
      Model  Accuracy  F1 Score  Precision   Recall
0       CNN   0.87680  0.873688   0.896331  0.85216
1  CNN-LSTM   0.87444  0.878234   0.852474  0.90560
